### ランダムフォレスト　パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 15)

In [4]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [5]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

### 検証するパラメータの設定

In [6]:
grid_param = {
    'n_estimators': [60, 80, 100, 120],
    'criterion': ['mse'],
    'max_depth': [i for i in range(1, 11, 2)],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [7]:
grid_param = {
    'n_estimators': [60, 70, 80, 90, 100],
    'criterion': ['mse'],
    'max_depth': [5, 6, 7, 9],
    'min_samples_split': [i for i in range(2, 11, 2)],
    'min_samples_leaf': [i for i in range(1, 11, 2)],
    'random_state': [42]
}


In [8]:
def rmse(y, y_pred):
    ret = np.sqrt(mean_squared_error(y, y_pred))
    print("RMSE=", ret)
    return ret

### グリッドサーチ

In [9]:
gs = GridSearchCV(estimator=RandomForestRegressor(**grid_param), param_grid=grid_param, scoring=make_scorer(mean_squared_error, greater_is_better=False), cv=5, return_train_score=False)

In [10]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion=['mse'],
                                             max_depth=[5, 6, 7, 9],
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[1, 3, 5, 7, 9],
                                             min_samples_split=[2, 4, 6, 8, 10],
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=[60, 70, 8...
                                             warm_start=False),
             iid='deprecated', n_jobs=None,
   

### 一番良かったパラメータで学習

In [11]:
gs.best_score_

-16131.259600819345

In [12]:
gs.best_params_

{'criterion': 'mse',
 'max_depth': 9,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100,
 'random_state': 42}

In [13]:
RF = RandomForestRegressor(**gs.best_params_)

In [14]:
RF = RF.fit(train_X, train_y)

In [15]:
RF.feature_importances_

array([6.21483346e-02, 2.31951797e-01, 4.41176774e-01, 2.07301698e-02,
       4.52844604e-02, 1.32014499e-03, 7.57446502e-03, 8.69576822e-03,
       1.47174796e-05, 0.00000000e+00, 1.41665453e-01, 3.79016467e-02,
       1.53626905e-03])

In [16]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]].columns),
    reverse=True)

[(0.441, 'beds'),
 (0.232, 'bedrooms'),
 (0.142, 'room_type_Shared room'),
 (0.062, 'bathrooms'),
 (0.045, 'number_of_reviews'),
 (0.038, 'property_type_num'),
 (0.021, 'host_response_rate'),
 (0.009, 'cancellation_policy_super_strict_30'),
 (0.008, 'cancellation_policy_strict'),
 (0.001, 'cancellation_policy_moderate'),
 (0.0, 'room_type_Private room'),
 (0.0, 'cancellation_policy_super_strict_60')]

In [17]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

### テストデータで予測

In [18]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [19]:
test_x.shape, test_y.shape

((11065, 13), (11065,))

In [20]:
pred_y = RF.predict(test_x)

In [21]:
np.sqrt(mean_squared_error(test_y, pred_y))

128.4719293510552

### 検証データで予測

In [22]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [23]:
valid.shape

(18528, 12)

In [24]:
# ID の保存
valid_pass = valid.id.values

In [25]:
valid_X = valid.iloc[:, 1:].values

In [26]:
# valid_X.describe()

In [27]:
valid_X.shape, train_X.shape

((18528, 11), (44295, 11))

In [28]:
pred_valid_y = RF.predict(valid_X)

In [29]:
pred_valid_y.shape

(18528,)

In [30]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [31]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [32]:
result_df.to_csv("./RF_5.csv", header=False)